In [1]:
import pandas as pd
import numpy as np
import sklearn
import re
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
# Load in the train and test datasets
train = pd.read_csv('../../../Documents/Datasets/titanic/train.csv')
test = pd.read_csv('../../../Documents/Datasets/titanic/test.csv')
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
full_data = [train, test]

# Give the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
    
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
    
train['CategoricalAge'] = pd.cut(train['Age'], 5)

# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search('([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
    
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
# Feature selection
train_labels = train['Survived']
drop_elements = ['Name_length', 'PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']

train = train.drop(drop_elements, axis = 1)
train = train.drop(['Survived', 'CategoricalAge', 'CategoricalFare'], axis = 1)

passenger_id = test['PassengerId']
test  = test.drop(drop_elements, axis = 1)

In [5]:
train.head(5)

,Pclass,Sex,Age,Parch,Fare,Embarked,Has_Cabin,FamilySize,IsAlone,Title
0,3,1,1,0,0,0,0,2,0,1
1,1,0,2,0,3,1,1,2,0,3
2,3,0,1,0,1,0,0,1,1,2
3,1,0,2,0,3,0,1,2,0,3
4,3,1,2,0,1,0,0,1,1,1


In [6]:
model = Sequential()
model.add(Dense(12,activation='elu',input_dim=(train.shape[1])))
model.add(Dense(15, activation='elu'))
model.add(Dense(15, activation='elu'))
model.add(Dense(15, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                132       
_________________________________________________________________
dense_2 (Dense)              (None, 15)                195       
_________________________________________________________________
dense_3 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_4 (Dense)              (None, 15)                240       
_________________________________________________________________
dropout_1 (Dropout)          (None, 15)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 16        
Total params: 823.0
Trainable params: 823.0
Non-trainable params: 0.0
_________________________________________________________________


In [7]:
sgd = SGD(lr=0.01)
train = np.array(train)
train_labels = np.array(train_labels)
test = np.array(test)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train, train_labels, batch_size=50, nb_epoch=200, verbose=1, validation_split = 0.1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 801 samples, validate on 90 samples
Epoch 1/200
801/801 [==============================] - 0s - loss: 0.7285 - acc: 0.5281 - val_loss: 0.6356 - val_acc: 0.6222
Epoch 2/200
801/801 [==============================] - 0s - loss: 0.6507 - acc: 0.6429 - val_loss: 0.5976 - val_acc: 0.6333
Epoch 3/200
801/801 [==============================] - 0s - loss: 0.6246 - acc: 0.6579 - val_loss: 0.5583 - val_acc: 0.7333
Epoch 4/200
801/801 [==============================] - 0s - loss: 0.5912 - acc: 0.7029 - val_loss: 0.5151 - val_acc: 0.8222
Epoch 5/200
801/801 [==============================] - 0s - loss: 0.5624 - acc: 0.7179 - val_loss: 0.4769 - val_acc: 0.8111
Epoch 6/200
801/801 [==============================] - 0s - loss: 0.5388 - acc: 0.7328 - val_loss: 0.4472 - val_acc: 0.8222
Epoch 7/200
801/801 [==============================] - 0s - loss: 0.5246 - acc: 0.7428 - val_loss: 0.4296 - val_acc: 0.8222
Epoch 8/200
801/801 [==============================] - 0s - loss: 0.4913 - acc: 0.7740 

801/801 [==============================] - 0s - loss: 0.4302 - acc: 0.8277 - val_loss: 0.3741 - val_acc: 0.8222
Epoch 67/200
801/801 [==============================] - 0s - loss: 0.4398 - acc: 0.8165 - val_loss: 0.3773 - val_acc: 0.8222
Epoch 68/200
801/801 [==============================] - 0s - loss: 0.4646 - acc: 0.8027 - val_loss: 0.3756 - val_acc: 0.8222
Epoch 69/200
801/801 [==============================] - 0s - loss: 0.4464 - acc: 0.8090 - val_loss: 0.3636 - val_acc: 0.8333
Epoch 70/200
801/801 [==============================] - 0s - loss: 0.4451 - acc: 0.8127 - val_loss: 0.3688 - val_acc: 0.8222
Epoch 71/200
801/801 [==============================] - 0s - loss: 0.4355 - acc: 0.8140 - val_loss: 0.3643 - val_acc: 0.8333
Epoch 72/200
801/801 [==============================] - 0s - loss: 0.4494 - acc: 0.8090 - val_loss: 0.3615 - val_acc: 0.8667
Epoch 73/200
801/801 [==============================] - 0s - loss: 0.4405 - acc: 0.8090 - val_loss: 0.3662 - val_acc: 0.8333
Epoch 74/200


801/801 [==============================] - 0s - loss: 0.4337 - acc: 0.8127 - val_loss: 0.3678 - val_acc: 0.8222
Epoch 132/200
801/801 [==============================] - 0s - loss: 0.4478 - acc: 0.7928 - val_loss: 0.3693 - val_acc: 0.8222
Epoch 133/200
801/801 [==============================] - 0s - loss: 0.4240 - acc: 0.8152 - val_loss: 0.3685 - val_acc: 0.8111
Epoch 134/200
801/801 [==============================] - 0s - loss: 0.4332 - acc: 0.8090 - val_loss: 0.3633 - val_acc: 0.8222
Epoch 135/200
801/801 [==============================] - 0s - loss: 0.4364 - acc: 0.8165 - val_loss: 0.3683 - val_acc: 0.8111
Epoch 136/200
801/801 [==============================] - 0s - loss: 0.4346 - acc: 0.8190 - val_loss: 0.3655 - val_acc: 0.8333
Epoch 137/200
801/801 [==============================] - 0s - loss: 0.4294 - acc: 0.8090 - val_loss: 0.3624 - val_acc: 0.8222
Epoch 138/200
801/801 [==============================] - 0s - loss: 0.4263 - acc: 0.8115 - val_loss: 0.3607 - val_acc: 0.8222
Epoch 

801/801 [==============================] - 0s - loss: 0.4143 - acc: 0.8240 - val_loss: 0.3506 - val_acc: 0.8444
Epoch 196/200
801/801 [==============================] - 0s - loss: 0.4487 - acc: 0.7903 - val_loss: 0.3533 - val_acc: 0.8778
Epoch 197/200
801/801 [==============================] - 0s - loss: 0.4330 - acc: 0.7978 - val_loss: 0.3563 - val_acc: 0.8111
Epoch 198/200
801/801 [==============================] - 0s - loss: 0.4383 - acc: 0.7953 - val_loss: 0.3570 - val_acc: 0.8667
Epoch 199/200
801/801 [==============================] - 0s - loss: 0.4217 - acc: 0.8027 - val_loss: 0.3721 - val_acc: 0.8111
Epoch 200/200
801/801 [==============================] - 0s - loss: 0.4492 - acc: 0.7978 - val_loss: 0.3638 - val_acc: 0.8000


In [10]:
pred = model.predict_classes(test)

 32/418 [=>............................] - ETA: 0s

In [11]:
import csv
with open('output.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames = ["PassengerId", "Survived"], delimiter = ',')
    writer.writeheader()
    writer = csv.writer(file, delimiter=',')
    writer.writerows(zip(passenger_id, pred.reshape(-1)))

In [11]:
# random forest
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)
forest = forest.fit(train, train_labels)
pred = forest.predict(test)

In [13]:
with open('output.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames = ["PassengerId", "Survived"], delimiter = ',')
    writer.writeheader()
    writer = csv.writer(file, delimiter=',')
    writer.writerows(zip(passenger_id, pred))